# SIMD Autovectorization in Numba

François-David Collin (CNRS, IMAG, Paul-Valéry Montpellier 3
University)  
Ghislain Durif (CNRS, LBMC)  
Monday, August 26, 2024

Most modern CPUs have support for instructions that apply the same
operation to multiple data elements simultaneously. These are called
“Single Instruction, Multiple Data” (SIMD) operations, and the LLVM
backend used by Numba can generate them in some cases to execute loops
more quickly. (This process is called “autovectorization.”)

For example, Intel processors have support for SIMD instruction sets
like:

-   SSE (128-bit inputs)
-   AVX (256-bit inputs)
-   AVX-512 (512-bit inputs, Skylake-X and later or Xeon Phi)

These wide instructions typically operate on as many values as will fit
into an input register. For AVX instructions, this means that either 8
float32 values or 4 float64 values can be processed as a single input.
As a result, the NumPy dtype that you use can potentially impact
performance to a greater degree than when SIMD is not in use.

In [2]:
import numpy as np
from numba import jit

It can be somewhat tricky to determine when LLVM has successfully
autovectorized a loop. The Numba team is working on exporting diagnostic
information to show where the autovectorizer has generated SIMD code.
For now, we can use a fairly crude approach of searching the assembly
language generated by LLVM for SIMD instructions.

It is also interesting to note what kind of SIMD is used on your system.
On x86_64, the name of the registers used indicates which level of SIMD
is in use:

-   SSE: `xmmX`
-   AVX/AVX2: `ymmX`
-   AVX-512: `zmmX`

where X is an integer.

**Note**: The method we use below to find SIMD instructions will only
work on Intel/AMD CPUs. Other platforms have entirely different assembly
language syntax for SIMD instructions.

In [3]:
def find_instr(func, keyword, sig=0, limit=5):
    count = 0
    for l in func.inspect_asm(func.signatures[sig]).split('\n'):
        if keyword in l:
            count += 1
            print(l)
            if count >= limit:
                break
    if count == 0:
        print('No instructions found')

## Basic SIMD

Let’s start with a simple function that returns the square difference
between two arrays, as you might write for a least-squares optimization:

In [4]:
@jit(nopython=True)
def sqdiff(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        out[i] = (x[i] - y[i])**2
    return out

In [5]:
x32 = np.linspace(1, 2, 10000, dtype=np.float32)
y32 = np.linspace(2, 3, 10000, dtype=np.float32)
sqdiff(x32, y32)

array([1.        , 0.99999976, 1.        , ..., 1.        , 1.0000002 ,
       1.        ], dtype=float32)

In [6]:
x64 = x32.astype(np.float64)
y64 = y32.astype(np.float64)
sqdiff(x64, y64)

array([1.        , 0.99999976, 1.        , ..., 1.        , 1.00000024,
       1.        ])

Numba has created two different implementations of the function, one for
`float32` 1-D arrays, and one for `float64` 1-D arrays:

In [7]:
sqdiff.signatures

[(Array(float32, 1, 'C', False, aligned=True),
  Array(float32, 1, 'C', False, aligned=True)),
 (Array(float64, 1, 'C', False, aligned=True),
  Array(float64, 1, 'C', False, aligned=True))]

This allows Numba (and LLVM) to specialize the use of the SIMD
instructions for each situation. In particular, using lower precision
floating point allows twice as many values to fit into a SIMD register.
We will see that for the same number of elements, the `float32`
calculation goes twice as fast:

In [8]:
%timeit sqdiff(x32, y32)
%timeit sqdiff(x64, y64)

2.04 μs ± 7.82 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.01 μs ± 72.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

We can check for SIMD instructions in both cases. (Due to the order of
compilation above, signature 0 is the `float32` implementation and
signature 1 is the `float64` implementation.)

In [9]:
print('float32:')
find_instr(sqdiff, keyword='subp', sig=0)
print('---\nfloat64:')
find_instr(sqdiff, keyword='subp', sig=1)

float32:
    vsubps  (%rax,%rdi,4), %ymm0, %ymm0
    vsubps  32(%rax,%rdi,4), %ymm1, %ymm1
    vsubps  64(%rax,%rdi,4), %ymm2, %ymm2
    vsubps  96(%rax,%rdi,4), %ymm3, %ymm3
    vsubps  128(%rax,%rdi,4), %ymm0, %ymm0
---
float64:
    vsubpd  (%rax,%rdi,8), %ymm0, %ymm0
    vsubpd  32(%rax,%rdi,8), %ymm1, %ymm1
    vsubpd  64(%rax,%rdi,8), %ymm2, %ymm2
    vsubpd  96(%rax,%rdi,8), %ymm3, %ymm3
    vsubpd  128(%rax,%rdi,8), %ymm0, %ymm0

In x86_64 assembly, SSE uses `subps` for “subtraction packed single
precision” (AVX uses `vsubps`), representing vector float32 operations.
The `subpd` instruction (AVX = `vsubpd`) stands for “subtraction packed
double precision”, representing float64 operations.

## SIMD and Division

In general, the autovectorizer cannot deal with branches inside loops,
although this is an area where LLVM is likely to improve in the future.
Your best bet for SIMD acceleration is to only have pure math operations
in the loop.

As a result, you would naturally assume a function like this would be
OK:

In [10]:
@jit(nopython=True)
def frac_diff1(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        out[i] = 2 * (x[i] - y[i]) / (x[i] + y[i])
    return out

In [11]:
frac_diff1(x32, y32)

array([-0.6666667 , -0.66662216, -0.66657776, ..., -0.400032  ,
       -0.40001604, -0.4       ], dtype=float32)

In [12]:
find_instr(frac_diff1, keyword='subp', sig=0)

No instructions found

`No instructions found`?!

The problem is that division by zero can behave in two different ways:

-   In Python, division by zero raises an exception.
-   In NumPy, division by zero results in a `NaN`, like in C.

By default, Numba `@jit` follows the Python convention, and
`@vectorize`/`@guvectorize` follow the NumPy convention. When following
the Python convention, a simple division operation `r = x / y` expands
out into something like:

``` python

if y == 0:
    raise ZeroDivisionError()
else:
    r = x / y
```

This branching code causes the autovectorizer to give up, and no SIMD to
be generated for our example above.

Fortunately, Numba allows you to override the “error model” of the
function if you don’t want a `ZeroDivisionError` to be raised:

In [13]:
@jit(nopython=True, error_model='numpy')
def frac_diff2(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        out[i] = 2 * (x[i] - y[i]) / (x[i] + y[i])
    return out

In [14]:
frac_diff2(x32, y32)

array([-0.6666667 , -0.66662216, -0.66657776, ..., -0.400032  ,
       -0.40001604, -0.4       ], dtype=float32)

In [15]:
find_instr(frac_diff2, keyword='subp', sig=0)

    vsubps  %ymm1, %ymm0, %ymm2
    vsubps  %ymm1, %ymm0, %ymm2
    vsubps  %ymm1, %ymm0, %ymm2

We have SIMD instructions again, but when we check the speed:

In [16]:
%timeit frac_diff2(x32, y32)
%timeit frac_diff2(x64, y64)

6.92 μs ± 33.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
6.46 μs ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

This is faster than the no-SIMD case, but there doesn’t seem to be a
speed benefit with `float32` inputs. What’s going on?

The remaining issue is very subtle. We can see it if we look at a
type-annotated version of the function:

In [17]:
frac_diff2.inspect_types(pretty=True)

/home/fradav/.micromamba/envs/miashs-hpc/lib/python3.12/site-packages/numba/core/annotations/pretty_annotate.py:7: FutureWarning:

The pretty_annotate functionality is experimental and might change API


If you expand out line 5 in the float32 version of the function, you
will see the following bit of Numba IR:

    $const28.2 = const(float, 2.0) :: float64
    $28.5 = getitem(value=x, index=i) :: float32
    $28.8 = getitem(value=y, index=i) :: float32
    $28.9 = $28.5 - $28.8 :: float32
    del $28.8
    del $28.5
    $28.10 = $const28.2 * $28.9 :: float64

Notice that the constant `2` has been typed as a float64 value. Later,
this causes the multiplication `2 * (x[i] - y[i]` to promote up to
float64, and then the rest of the calculation becomes float64. This is a
situation where Numba is being overly conservative (and should be fixed
at some point), but we can tweak this behavior by casting the constant
to the type we want:

In [18]:
@jit(nopython=True, error_model='numpy')
def frac_diff3(x, y):
    out = np.empty_like(x)
    dt = x.dtype # Cast the constant using the dtype of the input
    for i in range(x.shape[0]):
        # Could also use np.float32(2) to always use same type, regardless of input
        out[i] = dt.type(2) * (x[i] - y[i]) / (x[i] + y[i])
    return out

In [19]:
frac_diff3(x32, y32)

array([-0.6666667 , -0.66662216, -0.66657776, ..., -0.400032  ,
       -0.40001604, -0.4       ], dtype=float32)

In [20]:
%timeit frac_diff3(x32, y32)
%timeit frac_diff3(x64, y64)

2.63 μs ± 21.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
6.47 μs ± 35.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

Now our float32 version is nice and speedy (and 6x faster than what we
started with, if we only care about float32).

## SIMD and Reductions

The autovectorizer can also optimize reduction loops, but only with
permission. Normally, compilers are very careful not to reorder floating
point instructions because floating point arithmetic is approximate, so
mathematically allowed transformations do not always give the same
result. For example, it is not generally true for floating point numbers
that:

    (a + (b + c)) == ((a + b) + c)

For many situations, the round-off error that causes the difference
between the left and the right is not important, so changing the order
of additions is acceptable for a performance increase.

To allow reordering of operations, we need to tell Numba to enable
`fastmath` optimizations:

In [21]:
@jit(nopython=True)
def do_sum(A):
    acc = 0.
    # without fastmath, this loop must accumulate in strict order
    for x in A:
        acc += x**2
    return acc

@jit(nopython=True, fastmath=True)
def do_sum_fast(A):
    acc = 0.
    # with fastmath, the reduction can be vectorized as floating point
    # reassociation is permitted.
    for x in A:
        acc += x**2
    return acc

In [22]:
do_sum(x32)
find_instr(do_sum, keyword='mulp')  # look for vector multiplication

No instructions found

In [23]:
do_sum_fast(x32)
find_instr(do_sum_fast, keyword='mulp')

    vmulps  %xmm4, %xmm4, %xmm4
    vmulps  %xmm5, %xmm5, %xmm5
    vmulps  %xmm6, %xmm6, %xmm6
    vmulps  %xmm7, %xmm7, %xmm7

The fast version is 4x faster:

In [24]:
%timeit do_sum(x32)
%timeit do_sum_fast(x32)

11.4 μs ± 37.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
5.01 μs ± 3.87 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

## SIMD and Special Functions

If you follow the above guidelines, SIMD autovectorization will work for
all basic math operations (`+`,`-`,`*`,`\`), but generally will not work
for function calls in the loop, unless LLVM can inline the function and
there is only basic math in the function body.

However, we build Numba (if you get conda packages from Anaconda or
wheels from PyPI) using a patched version of LLVM that supports
vectorization of special math functions when Intel SVML (“Short Vector
Math Library”) is present. This library comes with the Intel compiler,
and is also freely redistributable. We’ve installed it in the current
conda environment using `conda install -c numba icc_rt`, as we can
verify here:

In [25]:
! conda list icc_rt

zsh:1: command not found: conda

Thanks to this library, we can still get SIMD vectorization in a
function like this:

In [26]:
SQRT_2PI = np.sqrt(2 * np.pi)

@jit(nopython=True, error_model='numpy', fastmath=True)
def kde(x, means, widths):
    '''Compute value of gaussian kernel density estimate.
    
    x - location of evaluation
    means - array of kernel means
    widths - array of kernel widths
    '''
    n = means.shape[0]
    acc = 0.
    for i in range(n):
        acc += np.exp( -0.5 * ((x - means[i]) / widths[i])**2 ) / widths[i]
    return acc / SQRT_2PI / n

In [27]:
# The distribution we are approximating is flat between -1 and 1, so we expect a KDE value of ~0.5 everywhere
means = np.random.uniform(-1, 1, size=10000)
# These widths are not selected in any reasonable way.  Consult your local statistician before approximating a PDF.
widths = np.random.uniform(0.1, 0.3, size=10000)

kde(0.4, means, widths)

0.48933504950629386

We can see that SIMD instructions were generated:

In [28]:
find_instr(kde, 'subp')

    vsubpd  (%rbx,%rsi,8), %ymm3, %ymm0
    vsubpd  32(%rbx,%rsi,8), %ymm3, %ymm1
    vsubpd  64(%rbx,%rsi,8), %ymm3, %ymm2
    vsubpd  96(%rbx,%rsi,8), %ymm3, %ymm3

We can also see that calls to the special Intel SVML functions for `exp`
were generated:

In [29]:
find_instr(kde, keyword='svml')

    movabsq $__svml_exp4, %rdi

In [30]:
%timeit kde(0.4, means, widths)

18.1 μs ± 79.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

If we recompile the function (which is possible since the `.py_func`
attribute holds the original Python function) with the extra flags to
allow division and reductions to work, this stops all autovectorization
of the loop:

In [31]:
slow_kde = jit(nopython=True)(kde.py_func)

slow_kde(0.4, means, widths)

0.4893350495062936

Note that we get a slightly different answer, both due to the different
order of operations, and the small differences in SVML `exp` compared to
the default `exp`. We also see that there is no SIMD or calls to SVML:

In [32]:
find_instr(slow_kde, keyword='subp')
print('---')
find_instr(slow_kde, keyword='svml')

No instructions found
---
No instructions found

And the function is much slower than the original:

In [33]:
%timeit kde(0.4, means, widths)
%timeit slow_kde(0.4, means, widths)

18.2 μs ± 81.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
82 μs ± 52.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

And only the SIMD vectorized version is faster than doing this in pure
NumPy:

In [34]:
def numpy_kde(x, means, widths):
    acc = (np.exp( -0.5 * ((x - means) / widths)**2 ) / widths).mean()
    # .mean() already divides by n
    return acc / SQRT_2PI

In [35]:
numpy_kde(0.4, means, widths)

np.float64(0.4893350495062938)

In [36]:
%timeit numpy_kde(0.4, means, widths)

37.5 μs ± 66.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Why is NumPy as fast as it is? In this case, it is because the Anaconda
build of NumPy uses MKL to accelerate (with SIMD and threads) many of
the individial ufuncs, so it is only when Numba can combine all the
operations together that the speed boost emerges.

Incidentally, although we wrote out the iteration for `kde` as a for
loop to highlight what was going on, you still get the benefit of SIMD
in Numba when compiling array expressions. We could have compiled
`numpy_kde` directly:

In [37]:
numba_numpy_kde = jit(nopython=True, error_model='numpy', fastmath=True)(numpy_kde)

numba_numpy_kde(0.4, means, widths)

0.4893350495062934

In [38]:
find_instr(numba_numpy_kde, keyword='subp')
print('---')
find_instr(numba_numpy_kde, keyword='svml')

    vsubpd  (%r14,%rsi,8), %ymm8, %ymm0
    vsubpd  (%r14,%rsi,8), %ymm8, %ymm0
---
    movabsq $__svml_exp4, %rdi
    movabsq $__svml_exp4, %rdi
    movabsq $__svml_exp4, %rdi
    movabsq $__svml_exp4, %rax

And it is nearly as fast as our manual looping version, and 2x faster
than NumPy alone:

In [39]:
%timeit kde(0.4, means, widths)
%timeit numba_numpy_kde(0.4, means, widths)
%timeit numpy_kde(0.4, means, widths)

18.1 μs ± 50.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
23.2 μs ± 91.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
37.4 μs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)